In [0]:
import os
os.chdir("drive/My Drive/SongGenerator/datasets/mousehead/songlyrics")

In [4]:
import pandas as pd
import numpy as np
songs = pd.read_csv("songdata.csv")
songs.head()

,artist,song,link,text
0,ABBA,Ahe's My Kind Of Girl,/a/abba/ahes+my+kind+of+girl_20598417.html,"Look at her face, it's a wonderful face \nAnd..."
1,ABBA,"Andante, Andante",/a/abba/andante+andante_20002708.html,"Take it easy with me, please \nTouch me gentl..."
2,ABBA,As Good As New,/a/abba/as+good+as+new_20003033.html,I'll never know why I had to go \nWhy I had t...
3,ABBA,Bang,/a/abba/bang_20598415.html,Making somebody happy is a question of give an...
4,ABBA,Bang-A-Boomerang,/a/abba/bang+a+boomerang_20002668.html,Making somebody happy is a question of give an...


In [5]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
sentences = songs["text"].str.cat(sep='\n').lower().split("\n")
tokenizer = Tokenizer()
tokenizer.fit_on_texts(sentences)

Using TensorFlow backend.


In [0]:
idx = {id : word for word, id in tokenizer.word_index.items()}
wordx = {word : id for word, id in tokenizer.word_index.items()}

In [0]:
idx[len(tokenizer.word_index)+1] = "<EOF>"
wordx["<EOF>"] = len(tokenizer.word_index)+1

In [0]:
idx[0] = "<PAD>"
wordx["<PAD>"] = 0

In [0]:
vocab_len = len(idx)

In [0]:
from keras.utils.np_utils import to_categorical

def get_output(input):
  temp = input.copy()
  temp.append(wordx["<EOF>"])
  return temp[1:]
def generator(sentences, batch_size = 64):
  for i in range(0, len(sentences), batch_size):
    if i+batch_size < len(sentences):
      curr_batch = sentences[i:i+batch_size]
      input_sequences = tokenizer.texts_to_sequences(curr_batch)
      output_sequences = list(map(get_output, input_sequences))
      input_sequences = pad_sequences(input_sequences, padding="post")
      output_sequences = pad_sequences(output_sequences, padding = "post")

      yield input_sequences, to_categorical(output_sequences, num_classes = vocab_len)


In [0]:
from keras.models import Model

from keras.models import Sequential
from keras.layers import Dense , Dropout , Lambda, Flatten, LSTM, Input, Embedding
from keras.layers import BatchNormalization
from keras.optimizers import Adam ,RMSprop
from keras.optimizers import SGD
from sklearn.model_selection import train_test_split
from keras import  backend as K
from keras.utils.np_utils import to_categorical


from keras.callbacks import EarlyStopping, TensorBoard, ModelCheckpoint
from keras.layers import BatchNormalization , MaxPooling2D, Activation

inputs = Input(shape=(None,))
emb = Embedding(vocab_len, 64)
emb_out = emb(inputs)
lstm = LSTM(64, return_sequences = True, return_state = True)
lstm_out, _, _ = lstm(emb_out)
soft = Dense(vocab_len, activation = "softmax")
out = soft(lstm_out)
model = Model(inputs, out)


In [0]:
from keras.losses import categorical_crossentropy
LR = 1e-2
model.compile(optimizer = Adam(LR), loss = categorical_crossentropy, metrics = ["accuracy"])

  

In [0]:
BATCH_SIZE = 64
train_gen = generator(sentences[:int(len(sentences)*0.9)], BATCH_SIZE)
val_gen = generator(sentences[int(len(sentences)*0.9):], BATCH_SIZE)
history = model.fit_generator(train_gen, steps_per_epoch= int(len(sentences)*0.9)//BATCH_SIZE, validation_data = val_gen, validation_steps= int(len(sentences)*0.1), epochs = 1, callbacks = [EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=7), ModelCheckpoint('best_model.h5', monitor='val_loss', mode='min', save_best_only=True)])



 2507/34139 [=>............................] - ETA: 4:07:25 - loss: 2.4445 - accuracy: 0.6486

In [0]:
state_h_in = Input(shape = (64,))
state_c_in  = Input(shape = (64,))
state_in = [state_h_in,state_c_in]
emb_out_pred = emb(inputs) 
out_pred, state_h_out, state_c_out = dec_lstm(emb_out_pred, initial_state = state_in)
out_pred = soft(out_pred)
model_second = Model([inputs] + state_in, [out_pred, state_h_out, state_c_out])
emb_out_pred2 = emb(inputs) 
out_pred2, state_h_out2, state_c_out2 = dec_lstm(emb_out_pred2)
out_pred2 = soft(out_pred2)
model_second = Model(inputs , [out_pred, state_h_out, state_c_out])



In [0]:
MAXLEN = 10
def predict_french(word):
  
  word = wordx[word])
  output = model.predict([word])
  output_seq = output[0]
  new_states = output[1:]
  init = new_states
  new_id = np.argmax(output_seq[0,-1,:])
  new_word = idx[new_id]
  word = np.array([[new_id]]) 
  stop = False
  sentence = 
  sentence += new_word + " "
  while not stop:
      output = model.predict([word] + init)
      output_seq = output[0]
      new_states = output[1:]
      new_id = np.argmax(output_seq[0,-1,:])
      new_word = idx[new_id]
      

      if (new_word ==  "<EOS>" or len(sentence.split()) == MAXLEN):
        stop = True
      else: 
        sentence += new_word + " "
      word = np.array([[new_id]]) 
      init = new_states
  return fre_sentence